In [3]:
#this notebook uses Chicago as a test set and SF and NY as a training set
import pandas as pd

In [4]:
msdf=pd.read_csv('michelin_star_restaurants2.csv')
msdf.head()

,categories,city,id,name,price,rating,review_count,zip_code
0,newamerican,Chicago,alinea-chicago,Alinea,$$$$,4.5,1542,60614
1,newamerican,Chicago,grace-chicago-3,Grace,$$$$,4.5,234,60661
2,newamerican,Chicago,42-grams-chicago,42 grams,$$$$,4.5,66,60640
3,newamerican,Chicago,acadia-chicago,Acadia,$$$$,4.0,334,60616
4,newamerican,Chicago,sixteen-chicago,Sixteen,$$$$,3.5,373,60611


In [5]:
nodf=pd.read_csv('no_star_restaurants2.csv')
nodf.head()
nodf=nodf[nodf.city!='Washington']

In [6]:
nodf['star']=0

In [7]:
star=pd.read_csv('michelin_list2.csv')
star.head()

,id,star
0,alinea-chicago,3
1,grace-chicago-3,3
2,42-grams-chicago,2
3,acadia-chicago,2
4,sixteen-chicago,2


In [8]:
result=pd.merge(msdf,star,how='left',on='id')

In [9]:
result[result['star'].isnull()]

,categories,city,id,name,price,rating,review_count,zip_code,star
43,modern_european,New York,bâtard-new-york,Bâtard,$$$$,4.0,172,10013,NaN
49,szechuan,New York,café-china-new-york-2,Café China,$$,3.5,655,10016,NaN
92,newamerican,Brooklyn,the-river-café-brooklyn-3,The River Café,$$$$,4.0,722,11201,NaN
96,german,New York,wallsé-new-york-2,Wallsé,$$$,4.0,195,10014,NaN
105,french,Palo Alto,baumé-palo-alto-5,Baumé,$$$$,4.0,306,94306,NaN
122,french,San Francisco,keiko-à-nob-hill-san-francisco-2,Keiko à Nob Hill,$$$$,4.5,312,94109,NaN


In [10]:
#manually inserting stars that didn't get in due to accents in characters
result['star'][43]=1
result['star'][49]=1
result['star'][92]=1
result['star'][96]=1
result['star'][105]=2
result['star'][122]=1

C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWa

In [11]:
result.shape

(148, 9)

In [12]:
trainer = result.append(nodf)

In [13]:
trainer.ix[trainer.price == '$$','price']=2
trainer.ix[trainer.price == '$$$','price']=3
trainer.ix[trainer.price == '$$$$','price']=4

In [14]:
trainer.shape

(271, 9)

In [15]:
trainer.head()

,categories,city,id,name,price,rating,review_count,zip_code,star
0,newamerican,Chicago,alinea-chicago,Alinea,4,4.5,1542,60614,3.0
1,newamerican,Chicago,grace-chicago-3,Grace,4,4.5,234,60661,3.0
2,newamerican,Chicago,42-grams-chicago,42 grams,4,4.5,66,60640,2.0
3,newamerican,Chicago,acadia-chicago,Acadia,4,4.0,334,60616,2.0
4,newamerican,Chicago,sixteen-chicago,Sixteen,4,3.5,373,60611,2.0


In [16]:
trainer['rating']=pd.to_numeric(trainer['rating'])
trainer['price']=pd.to_numeric(trainer['price'])
trainer['review_count']=pd.to_numeric(trainer['review_count'])
trainer['star']=pd.to_numeric(trainer['star'])

In [17]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
ss=StandardScaler()
le=LabelEncoder()
#label encoded category data
le.fit(list(trainer['categories'].values))
trainer['categories']=le.transform(trainer['categories'].values)

In [18]:
trainer.categories.unique()

array([24,  1, 10, 14, 19, 16, 21, 34, 29, 38, 26, 32, 22, 18, 35, 15, 33,
       37,  3, 27, 12, 30, 23, 36, 28,  7,  0,  8,  2, 11,  4, 25, 17, 13,
       41, 31,  5,  6, 20,  9, 39, 40], dtype=int64)

In [19]:
trainer.ix[trainer.city=='Chicago'].shape

(54, 9)

In [20]:
trainer.ix[trainer.city!='Chicago'].shape

(217, 9)

In [21]:
y_train=trainer.ix[trainer.city!='Chicago'][['star']]
X_train=trainer.ix[trainer.city!='Chicago'][['categories','price','rating','review_count']]

In [22]:
y_test=trainer.ix[trainer.city=='Chicago'][['star']]
X_test=trainer.ix[trainer.city=='Chicago'][['categories','price','rating','review_count']]

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
svc = SVC()
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
svc.fit(X_train,y_train)
from sklearn.linear_model  import LogisticRegression
lr=LogisticRegression().fit(X_train,y_train)
y_pred_lr=lr.predict(X_test)
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier()
et.fit(X_train,y_train)
y_pred_et=et.predict(X_test)

C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
C:\Users\Ouisu\Anaconda2\lib\site-packages\sklearn\svm\base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
C:\Users\Ouisu\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ouisu\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [24]:
y_pred_rf=rf.predict(X_test)
y_pred_svc=svc.predict(X_test)

In [25]:
from sklearn.ensemble import GradientBoostingClassifier
gbc=GradientBoostingClassifier()
gbc.fit(X_train,y_train)
y_pred_gbc=gbc.predict(X_test)

C:\Users\Ouisu\Anaconda2\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [26]:
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier()
dt.fit(X_train,y_train)
y_pred_dt=dt.predict(X_test)

In [27]:
zip(trainer.ix[trainer.city=='Chicago']['name'],y_pred_rf, y_pred_svc,y_pred_gbc,y_pred_dt,y_pred_et,trainer.ix[trainer.city=='Chicago']['star'])

[('Alinea', 1.0, 1.0, 3.0, 3.0, 3.0, 3.0),
 ('Grace', 1.0, 1.0, 1.0, 1.0, 1.0, 3.0),
 ('42 grams', 1.0, 1.0, 1.0, 1.0, 1.0, 2.0),
 ('Acadia', 1.0, 0.0, 1.0, 1.0, 1.0, 2.0),
 ('Sixteen', 2.0, 1.0, 1.0, 2.0, 2.0, 2.0),
 ('Blackbird', 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ('Boka', 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ("Dusek's Board & Beer", 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ('EL Ideas', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Elizabeth Restaurant', 2.0, 1.0, 2.0, 2.0, 1.0, 1.0),
 ('Everest', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Goosefoot', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Longman & Eagle', 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ('Moto', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Naha', 2.0, 2.0, 1.0, 2.0, 2.0, 1.0),
 ('North Pond', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Parachute', 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ('Schwa', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Sepia', 0.0, 1.0, 0.0, 0.0, 0.0, 1.0),
 ('Spiaggia', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Topolobampo', 1.0, 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Tru', 2.0, 1.0, 2.0, 2.0, 1.0, 1.0),
 ('

In [28]:
from sklearn.metrics import accuracy_score
print accuracy_score(y_pred_rf,trainer.ix[trainer.city=='Chicago']['star'])
print accuracy_score(y_pred_svc,trainer.ix[trainer.city=='Chicago']['star'])
print accuracy_score(y_pred_gbc,trainer.ix[trainer.city=='Chicago']['star'])
print accuracy_score(y_pred_dt,trainer.ix[trainer.city=='Chicago']['star'])
print accuracy_score(y_pred_lr,trainer.ix[trainer.city=='Chicago']['star'])
print accuracy_score(y_pred_et,trainer.ix[trainer.city=='Chicago']['star'])

0.703703703704
0.333333333333
0.611111111111
0.648148148148
0.740740740741
0.777777777778


In [29]:
y=trainer['star']
X=trainer[['categories','price','rating','review_count']]
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
svc = SVC().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
gbc=GradientBoostingClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier(max_depth = 3, min_samples_split = 2)
dt.fit(X_train, y_train)
from sklearn.linear_model  import LogisticRegression
lr=LogisticRegression().fit(X_train,y_train)
y_pred_lr=lr.predict(X_test)
et=ExtraTreesClassifier().fit(X_train,y_train)
y_pred_et=et.predict(X_test)

print svc.score(X_test, y_test)
print rf.score(X_test, y_test)
print gbc.score(X_test, y_test)
print dt.score(X_test, y_test)
print lr.score(X_test, y_test)
print et.score(X_test, y_test)

0.467889908257
0.651376146789
0.633027522936
0.697247706422
0.706422018349
0.614678899083


In [30]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(class_weight='balanced', n_jobs=-1)
et.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight='balanced',
           criterion='gini', max_depth=None, max_features='auto',
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [31]:
feature_importances = pd.DataFrame(zip(dt.feature_importances_,
                               rf.feature_importances_,
                               gbc.feature_importances_,
                               et.feature_importances_),
                                   index = trainer[['categories','price','rating','review_count']].columns,
                                    columns=['dt_importance',
                                    'rf_importance','gbc_importance',
                                    'et_importance']).sort_values('rf_importance',
                                                                        ascending=False)
feature_importances.head()

,dt_importance,rf_importance,gbc_importance,et_importance
review_count,0.134506,0.418228,0.658249,0.470414
price,0.658423,0.336618,0.127881,0.236057
categories,0.207071,0.196739,0.182578,0.193588
rating,0.000000,0.048415,0.031292,0.099940


In [32]:
#mapped neighboring cities to major cities
trainer.ix[trainer.city=='Brooklyn','city']='New York'
trainer.ix[trainer.city=='Long Island City','city']='New York'
trainer.ix[trainer.city=='Williamsburg','city']='New York'
trainer.ix[trainer.city=='Clinton Hill','city']='New York'
trainer.ix[trainer.city=='Yountville','city']='San Francisco'
trainer.ix[trainer.city=='Los Gatos','city']='San Francisco'
trainer.ix[trainer.city=='St. Helena','city']='San Francisco'
trainer.ix[trainer.city=='Palo Alto','city']='San Francisco'
trainer.ix[trainer.city=='Oakland','city']='San Francisco'
trainer.ix[trainer.city=='Rutherford','city']='San Francisco'
trainer.ix[trainer.city=='Mountain View','city']='San Francisco'
trainer.ix[trainer.city=='Forestville','city']='San Francisco'
trainer.ix[trainer.city=='Napa','city']='San Francisco'
trainer.ix[trainer.city=='Healdsburg','city']='San Francisco'
trainer.ix[trainer.city=='Saratoga','city']='San Francisco'
trainer.ix[trainer.city=='Burlingame','city']='San Francisco'
trainer.ix[trainer.city=='Calistoga','city']='San Francisco'
trainer.ix[trainer.city=='San Mateo','city']='San Francisco'
trainer.ix[trainer.city=='Saint Helena','city']='San Francisco'
trainer.ix[trainer.city=='Bodega Bay','city']='San Francisco'
trainer.ix[trainer.city=='Woodside','city']='San Francisco'
trainer.ix[trainer.city=='Sausalito','city']='San Francisco'
trainer.ix[trainer.city=='Berkeley','city']='San Francisco'
trainer.ix[trainer.city=='Albany','city']='San Francisco'

In [33]:
print trainer.ix[trainer.city=='Chicago']['rating'].mean()
print trainer.ix[trainer.city=='San Francisco']['rating'].mean()
print trainer.ix[trainer.city=='New York']['rating'].mean()

4.06481481481
4.05913978495
4.00403225806


In [34]:
#standardizing ratings based on average of respective cities
trainer.ix[trainer.city=='New York','rating']= \
(trainer.ix[trainer.city=='New York','rating']-trainer.ix[trainer.city=='New York','rating'].mean())/ \
trainer.ix[trainer.city=='New York','rating'].std()

In [35]:
trainer.ix[trainer.city=='San Francisco','rating']= \
(trainer.ix[trainer.city=='San Francisco','rating']-trainer.ix[trainer.city=='San Francisco','rating'].mean())/ \
trainer.ix[trainer.city=='San Francisco','rating'].std()

In [36]:
trainer.ix[trainer.city=='Chicago','rating']= \
(trainer.ix[trainer.city=='Chicago','rating']-trainer.ix[trainer.city=='Chicago','rating'].mean())/ \
trainer.ix[trainer.city=='Chicago','rating'].std()

In [37]:
trainer.ix[trainer.city=='New York','review_count']= \
(trainer.ix[trainer.city=='New York','review_count']-trainer.ix[trainer.city=='New York','review_count'].mean())/ \
trainer.ix[trainer.city=='New York','review_count'].std()

trainer.ix[trainer.city=='San Francisco','review_count']= \
(trainer.ix[trainer.city=='San Francisco','review_count']-trainer.ix[trainer.city=='San Francisco','review_count'].mean())/ \
trainer.ix[trainer.city=='San Francisco','review_count'].std()

trainer.ix[trainer.city=='Chicago','review_count']= \
(trainer.ix[trainer.city=='Chicago','review_count']-trainer.ix[trainer.city=='Chicago','review_count'].mean())/ \
trainer.ix[trainer.city=='Chicago','review_count'].std()

trainer.ix[trainer.city=='New York','price']= \
(trainer.ix[trainer.city=='New York','price']-trainer.ix[trainer.city=='New York','price'].mean())/ \
trainer.ix[trainer.city=='New York','price'].std()

trainer.ix[trainer.city=='San Francisco','price']= \
(trainer.ix[trainer.city=='San Francisco','price']-trainer.ix[trainer.city=='San Francisco','price'].mean())/ \
trainer.ix[trainer.city=='San Francisco','price'].std()

trainer.ix[trainer.city=='Chicago','price']= \
(trainer.ix[trainer.city=='Chicago','price']-trainer.ix[trainer.city=='Chicago','price'].mean())/ \
trainer.ix[trainer.city=='Chicago','price'].std()

In [38]:
y=trainer['star']
X=trainer[['categories','price','rating','review_count']]
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)
svc = SVC().fit(X_train, y_train)
rf = RandomForestClassifier().fit(X_train, y_train)
gbc=GradientBoostingClassifier().fit(X_train, y_train)
dt = DecisionTreeClassifier(max_depth = 3, min_samples_split = 2)
dt.fit(X_train, y_train)
print svc.score(X_test, y_test)
print rf.score(X_test, y_test)
print gbc.score(X_test, y_test)
print dt.score(X_test, y_test)

0.642201834862
0.669724770642
0.642201834862
0.706422018349


In [39]:
trainer['star'].value_counts()/len(trainer['star'])

0.0    0.453875
1.0    0.424354
2.0    0.073801
3.0    0.047970
Name: star, dtype: float64

In [53]:
feature_importances = pd.DataFrame(zip(dt.feature_importances_,
                               rf.feature_importances_,
                               gbc.feature_importances_,
                               et.feature_importances_),
                                   index = trainer[['categories','price','rating','review_count']].columns,
                                    columns=['dt_importance',
                                    'rf_importance','gb_importance',
                                    'et_importance']).sort_values('rf_importance',
                                                                        ascending=False)
feature_importances.head()

,dt_importance,rf_importance,gb_importance,et_importance
review_count,0.165311,0.403993,0.532591,0.470414
price,0.651389,0.302373,0.187523,0.236057
categories,0.123354,0.148029,0.190497,0.193588
rating,0.059946,0.145605,0.089388,0.099940


In [41]:
zip(trainer.ix[trainer.city=='Chicago']['name'],y_pred_rf, y_pred_svc,y_pred_gbc,y_pred_dt,trainer.ix[trainer.city=='Chicago']['star'])

[('Alinea', 1.0, 1.0, 3.0, 3.0, 3.0),
 ('Grace', 1.0, 1.0, 1.0, 1.0, 3.0),
 ('42 grams', 1.0, 1.0, 1.0, 1.0, 2.0),
 ('Acadia', 1.0, 0.0, 1.0, 1.0, 2.0),
 ('Sixteen', 2.0, 1.0, 1.0, 2.0, 2.0),
 ('Blackbird', 0.0, 1.0, 0.0, 0.0, 1.0),
 ('Boka', 0.0, 1.0, 0.0, 0.0, 1.0),
 ("Dusek's Board & Beer", 0.0, 1.0, 0.0, 0.0, 1.0),
 ('EL Ideas', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Elizabeth Restaurant', 2.0, 1.0, 2.0, 2.0, 1.0),
 ('Everest', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Goosefoot', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Longman & Eagle', 0.0, 1.0, 0.0, 0.0, 1.0),
 ('Moto', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Naha', 2.0, 2.0, 1.0, 2.0, 1.0),
 ('North Pond', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Parachute', 0.0, 1.0, 0.0, 0.0, 1.0),
 ('Schwa', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Sepia', 0.0, 1.0, 0.0, 0.0, 1.0),
 ('Spiaggia', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Topolobampo', 1.0, 1.0, 1.0, 1.0, 1.0),
 ('Tru', 2.0, 1.0, 2.0, 2.0, 1.0),
 ('Herb', 1.0, 1.0, 1.0, 1.0, 0.0),
 ('Untitled Supper Club', 1.0, 1.0, 1.0, 1.0, 0.0),
 ('Gilt Bar', 0.0, 1.0, 0

In [49]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_gbc))
print(classification_report(y_test, y_pred_dt))
print(classification_report(y_test, y_pred_et))
print(classification_report(y_test, y_pred_rf))

             precision    recall  f1-score   support

        0.0       0.76      0.86      0.81        51
        1.0       0.65      0.70      0.67        47
        2.0       0.00      0.00      0.00         7
        3.0       0.00      0.00      0.00         4

avg / total       0.63      0.71      0.67       109



ValueError: Found arrays with inconsistent numbers of samples: [ 54 109]

In [50]:
trainer['star'].value_counts()/len(trainer['star'])

0.0    0.453875
1.0    0.424354
2.0    0.073801
3.0    0.047970
Name: star, dtype: float64